In [47]:

import sys, os
#sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))

import numpy as np
import matplotlib.pyplot as plt
import Derivate_coeff as dc
import Utilities as ut
import Fisher_Matrix as fm
from Fisher_Matrix import FisherAnalysisConfig
import Derivatives as dv
import concurrent.futures

The minimal code to compute the fisher is this:

In [48]:
# rewrite the cell as a comment and explain all the entries


"""
        Initialize the FisherAnalysisConfig class with the necessary parameters.

        Parameters:
        - file_path: Path to the fiducial.txt file.
        - fields: List of fields used for the analysis.
        - vars: List of variables for the analysis.
        - epsilon: List of perturbation sizes for derivatives.
        - fid_values: List of fiducial values for the parameters.
        - noisedict: Dictionary containing noise information.
        - base_path: Base path to the data files.
        - directory_structure: Directory structure template for data files.
        - fskies: List of sky fractions for each field.
        - lmax: Maximum multipole moment. IT MUST COINCIDE WITH THE MAXIMUM MULTIPOLE MOMENT IN THE DATA FILES.
        - lmin: Minimum multipole moment.
        - excluded_probes_euclid: List of probes to be excluded from the Euclid analysis.
        - lminimi: List of minimum multipole moments for each field.
        - lmassimi: List of maximum multipole moments for each field.
        - fsky: Optional sky fraction (default is None).
        """

# Usage example
# config = fm.FisherAnalysisConfig(
#     file_path='/home/guglielmo/Desktop/Codes/Euclid_Spectra/GR_3k/Omegab/unlensed/fiducial.txt',
#     fields=fields,
#     vars=vars,
#     epsilon=epsilon,
#     fid_values=fid_values,
#     noisedict=noisedict,
#     base_path='/home/guglielmo/Desktop/Codes',
#     directory_structure='guglielmo/GR_test2',
#     fskies=fskies,
#     lmax=1500,
#     lmin=2,
#     excluded_probes_euclid=excluded_probes_euclid,
#     lminimi=lminimi,
#     lmassimi=lmassimi
# )

# fisher, covariance_fish_euclid, errori_euclid = compute_fisher_analysis(config)

'\n        Initialize the FisherAnalysisConfig class with the necessary parameters.\n\n        Parameters:\n        - file_path: Path to the fiducial.txt file.\n        - fields: List of fields used for the analysis.\n        - vars: List of variables for the analysis.\n        - epsilon: List of perturbation sizes for derivatives.\n        - fid_values: List of fiducial values for the parameters.\n        - noisedict: Dictionary containing noise information.\n        - base_path: Base path to the data files.\n        - directory_structure: Directory structure template for data files.\n        - fskies: List of sky fractions for each field.\n        - lmax: Maximum multipole moment. IT MUST COINCIDE WITH THE MAXIMUM MULTIPOLE MOMENT IN THE DATA FILES.\n        - lmin: Minimum multipole moment.\n        - excluded_probes_euclid: List of probes to be excluded from the Euclid analysis.\n        - lminimi: List of minimum multipole moments for each field.\n        - lmassimi: List of maxim

In [49]:

# define the ells of the spectra
lmax = 1500 #T MUST COINCIDE WITH THE MAXIMUM MULTIPOLE MOMENT IN THE DATA FILES
lmin = 2

# cosmological parameters fiducial values
As_fid= 2.12605e-9
ns_fid=0.96
H0_fid=67.0
ombh2_fid=0.05*0.67**2
omch2_fid=0.268565*0.67**2
tau_fid= 0.058
logAss_fid = np.log(As_fid*1e10)
omh2_fid=ombh2_fid+omch2_fid
sigma8_fid = 0.816

b1_fid = 1.0997727037892875
b2_fid = 1.220245876862528
b3_fid = 1.2723993083933989
b4_fid = 1.316624471897739
b5_fid = 1.35812370570578
b6_fid = 1.3998214171814918
b7_fid = 1.4446452851824907
b8_fid = 1.4964959071110084
b9_fid = 1.5652475842498528
b10_fid = 1.7429859437184225

#define the fields involved (not the probes!)
fields = ['T', 'E', 'P', 'W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10']

#define the fsky for each field
fskies = [0.7, 0.7, 0.7, 0.36, 0.36, 0.36, 0.36, 0.36, 0.36, 0.36, 0.36, 0.36, 0.36]



#define the parameters involved
vars = ["Omegam","Omegab","sigma8","ns","h0",'b1', 'b2', 'b3', 'b4','b5', 'b6','b7','b8','b9','b10']


fid_values = [0.32, 0.05, sigma8_fid, ns_fid, H0_fid, b1_fid, b2_fid, b3_fid,b4_fid,b5_fid,b6_fid,b7_fid,b8_fid,b9_fid,b10_fid]





# define the perturbation array
step = 0.01
epsilon = [-10*step,0*step,10*step]




#Define the noise


filename = "/home/guglielmo/Desktop/Codes/Euclid_Forecast_code/new_noises.npz"
data = np.load(filename, allow_pickle=True,encoding="latin1")
check = data['planck'].item()
check2 = data['euclid'].item()
N_G = check2['G5-G5']
N_L = check2['L4-L4']
N_P_adimensionale = check['E-E']
N_T_adimensionale = check['T-T']
nlkk= np.loadtxt(os.path.join( '/home/guglielmo/Desktop/Codes/nlkk.dat'), usecols=1)
N_K = nlkk[:lmax+1] #lensing convergence noise

#Final dictionary of noises
noisedict = {'TxT':N_T_adimensionale, 'ExE':N_P_adimensionale, "PxP": N_K, 'W1xW1': N_G, 'W2xW2': N_G, 'W3xW3': N_G, 'W4xW4': N_G, 'W5xW5': N_G, 'W6xW6': N_G, 'W7xW7': N_G, 'W8xW8': N_G, 'W9xW9': N_G, 'W10xW10': N_G}




In [50]:
# Define dictionary of minimum ell for each field

lminimi = {'TxT':2,
'TxE':2,
'TxP':8,
'TxW1':10,
'TxW2':10,
'TxW3':10,
'TxW4':10,
'TxW5':10,
'TxW6':10,
'TxW7':10,
'TxW8':10,
'TxW9':10,
'TxW10':10,
'ExE':2,
'ExP':8,
'ExW1':10,
'ExW2':10,
'ExW3':10,
'ExW4':10,
'ExW5':10,
'ExW6':10,
'ExW7':10,
'ExW8':10,
'ExW9':10,
'ExW10':10,
'PxP':8,
'PxW1':10,
'PxW2':10,
'PxW3':10,
'PxW4':10,
'PxW5':10,
'PxW6':10,
'PxW7':10,
'PxW8':10,
'PxW9':10,
'PxW10':10,
'W1xW1':10,
'W1xW2':10,
'W1xW3':10,
'W1xW4':10,
'W1xW5':10,
'W1xW6':10,
'W1xW7':10,
'W1xW8':10,
'W1xW9':10,
'W1xW10':10,
'W2xW2':10,
'W2xW3':10,
'W2xW4':10,
'W2xW5':10,
'W2xW6':10,
'W2xW7':10,
'W2xW8':10,
'W2xW9':10,
'W2xW10':10,
'W3xW3':10,
'W3xW4':10,
'W3xW5':10,
'W3xW6':10,
'W3xW7':10,
'W3xW8':10,
'W3xW9':10,
'W3xW10':10,
'W4xW4':10,
'W4xW5':10,
'W4xW6':10,
'W4xW7':10,
'W4xW8':10,
'W4xW9':10,
'W4xW10':10,
'W5xW5':10,
'W5xW6':10,
'W5xW7':10,
'W5xW8':10,
'W5xW9':10,
'W5xW10':10,
'W6xW6':10,
'W6xW7':10,
'W6xW8':10,
'W6xW9':10,
'W6xW10':10,
'W7xW7':10,
'W7xW8':10,
'W7xW9':10,
'W7xW10':10,
'W8xW8':10,
'W8xW9':10,
'W8xW10':10,
'W9xW9':10,
'W9xW10':10,
'W10xW10':10,
}

In [51]:

# Define the maximum multipole moments for each field


lmassimi = {'TxT':1500,
'TxE':1500,
'TxP':400,
'TxW1':200,
'TxW2':200,
'TxW3':200,
'TxW4':200,
'TxW5':200,
'TxW6':200,
'TxW7':200,
'TxW8':200,
'TxW9':200,
'TxW10':200,
'ExE':1500,
'ExP':400,
'ExW1':200,
'ExW2':200,
'ExW3':200,
'ExW4':200,
'ExW5':200,
'ExW6':200,
'ExW7':200,
'ExW8':200,
'ExW9':200,
'ExW10':200,
'PxP':400,
'PxW1':400,
'PxW2':400,
'PxW3':400,
'PxW4':400,
'PxW5':400,
'PxW6':400,
'PxW7':400,
'PxW8':400,
'PxW9':400,
'PxW10':400,
'W1xW1':750,
'W1xW2':750,
'W1xW3':750,
'W1xW4':750,
'W1xW5':750,
'W1xW6':750,
'W1xW7':750,
'W1xW8':750,
'W1xW9':750,
'W1xW10':750,
'W2xW2':750,
'W2xW3':750,
'W2xW4':750,
'W2xW5':750,
'W2xW6':750,
'W2xW7':750,
'W2xW8':750,
'W2xW9':750,
'W2xW10':750,
'W3xW3':750,
'W3xW4':750,
'W3xW5':750,
'W3xW6':750,
'W3xW7':750,
'W3xW8':750,
'W3xW9':750,
'W3xW10':750,
'W4xW4':750,
'W4xW5':750,
'W4xW6':750,
'W4xW7':750,
'W4xW8':750,
'W4xW9':750,
'W4xW10':750,
'W5xW5':750,
'W5xW6':750,
'W5xW7':750,
'W5xW8':750,
'W5xW9':750,
'W5xW10':750,
'W6xW6':750,
'W6xW7':750,
'W6xW8':750,
'W6xW9':750,
'W6xW10':750,
'W7xW7':750,
'W7xW8':750,
'W7xW9':750,
'W7xW10':750,
'W8xW8':750,
'W8xW9':750,
'W8xW10':750,
'W9xW9':750,
'W9xW10':750,
'W10xW10':750,
}

In [52]:
# Here we define the probes that we want to exclude from the Fisher analysis. Here I excluded all the CMB probes and cross-correlations between CMB and LSS probes. In this way I obtain a Fisher matrix that contains only the information from the LSS probes.
excluded_probes_euclid = ['TxT',
'TxE',
'TxP',
'TxW1',
'TxW2',
'TxW3',
'TxW4',
'TxW5',
'TxW6',
'TxW7',
'TxW8',
'TxW9',
'TxW10',
'ExE',
'ExP',
'ExW1',
'ExW2',
'ExW3',
'ExW4',
'ExW5',
'ExW6',
'ExW7',
'ExW8',
'ExW9',
'ExW10',
'PxP',
'PxW1',
'PxW2',
'PxW3',
'PxW4',
'PxW5',
'PxW6',
'PxW7',
'PxW8',
'PxW9',
'PxW10',
 ]

In [53]:
config = fm.FisherAnalysisConfig(
    file_path='/home/guglielmo/Desktop/Codes/Euclid_Spectra/GR_3k/Omegab/unlensed/fiducial.txt',
    fields=fields,
    vars=vars,
    epsilon=epsilon,
    fid_values=fid_values,
    noisedict=noisedict,
    base_path='/home/guglielmo/Desktop/Codes',
    directory_structure='guglielmo/GR_test4',
    fskies=fskies,
    lmax=1500,   #IT MUST COINCIDE WITH THE MAXIMUM MULTIPOLE MOMENT IN THE DATA FILES
    lmin=2,
    excluded_probes_euclid=excluded_probes_euclid,
    lminimi=lminimi,
    lmassimi=lmassimi
)

fisher, covariance_fish_euclid, errori_euclid = fm.compute_fisher_analysis(config)

''' This fisher computation is the default one and it computes the Fisher matrix using:
- CMB lensing as lensing convergence
- The fskies are computed for each probe and the fksy for a cross-correlation is the minimum between the two fskies
- The derivatives are computed using the Polynomial method (more time consuming but more accurate)
'''



Processing Data: 100%|██████████| 4095/4095 [00:13<00:00, 310.00it/s]


Processing Omegam...


100%|██████████| 1499/1499 [00:13<00:00, 111.76it/s]


Processing Omegab...


100%|██████████| 1499/1499 [00:13<00:00, 113.25it/s]


...

100%|██████████| 1499/1499 [00:14<00:00, 104.83it/s]


Processing b10...


100%|██████████| 1499/1499 [00:14<00:00, 104.77it/s]


0 0 ['T' 'T' 'T' 'T']
0 1 ['T' 'T' 'T' 'E']
0 2 ['T' 'T' 'T' 'P']
0 3 ['T' 'T' 'T' 'W1']
0 4 ['T' 'T' 'T' 'W2']
0 5 ['T' 'T' 'T' 'W3']
0 6 ['T' 'T' 'T' 'W4']
0 7 ['T' 'T' 'T' 'W5']
0 8 ['T' 'T' 'T' 'W6']
0 9 ['T' 'T' 'T' 'W7']
0 10 ['T' 'T' 'T' 'W8']
0 11 ['T' 'T' 'T' 'W9']
0 12 ['T' 'T' 'T' 'W10']
0 13 ['T' 'T' 'E' 'E']
0 14 ['T' 'T' 'E' 'P']
0 15 ['T' 'T' 'E' 'W1']
0 16 ['T' 'T' 'E' 'W2']
0 17 ['T' 'T' 'E' 'W3']
0 18 ['T' 'T' 'E' 'W4']
0 19 ['T' 'T' 'E' 'W5']
0 20 ['T' 'T' 'E' 'W6']
0 21 ['T' 'T' 'E' 'W7']
0 22 ['T' 'T' 'E' 'W8']
0 23 ['T' 'T' 'E' 'W9']
0 24 ['T' 'T' 'E' 'W10']
0 25 ['T' 'T' 'P' 'P']
0 26 ['T' 'T' 'P' 'W1']
0 27 ['T' 'T' 'P' 'W2']
0 28 ['T' 'T' 'P' 'W3']
0 29 ['T' 'T' 'P' 'W4']
0 30 ['T' 'T' 'P' 'W5']
0 31 ['T' 'T' 'P' 'W6']
0 32 ['T' 'T' 'P' 'W7']
0 33 ['T' 'T' 'P' 'W8']
0 34 ['T' 'T' 'P' 'W9']
0 35 ['T' 'T' 'P' 'W10']
0 36 ['T' 'T' 'W1' 'W1']
0 37 ['T' 'T' 'W1' 'W2']
0 38 ['T' 'T' 'W1' 'W3']
0 39 ['T' 'T' 'W1' 'W4']
0 40 ['T' 'T' 'W1' 'W5']
0 41 ['T' 'T' 'W1' 'W6']

' This fisher computation is the default one and it computes the Fisher matrix using:\n- CMB lensing as lensing convergence\n- The fskies are computed for each probe and the fksy for a cross-correlation is the minimum between the two fskies\n- The derivatives are computed using the Polynomial method (more time consuming but more accurate)\n'

In [55]:
#Now print the errors and the relative errors (sigma/fiducial)
errori_finali_euclid = ut.print_errors(vars, fid_values, errori_euclid)

DISCLAIMER: Omegab is ombh2 and Omegac is omch2
absolute sigma Omegam = 0.0077315830276031465
relative sigma Omegam = 0.02416119696125983
absolute sigma Omegab = 0.002900421821084963
relative sigma Omegab = 0.058008436421699255
absolute sigma sigma8 = 0.012158028984863498
relative sigma sigma8 = 0.01489954532458762
absolute sigma ns = 0.02883184658465595
relative sigma ns = 0.03003317352568328
absolute sigma h0 = 2.6721452276781847
relative sigma h0 = 0.03988276459221171
absolute sigma b1 = 0.02354198051388332
relative sigma b1 = 0.02140622369765042
absolute sigma b2 = 0.027364612702911118
relative sigma b2 = 0.02242549081441723
absolute sigma b3 = 0.02819052196841727
relative sigma b3 = 0.022155404975826472
absolute sigma b4 = 0.02880580269056918
relative sigma b4 = 0.021878525961961996
absolute sigma b5 = 0.029334137181762927
relative sigma b5 = 0.02159901712820687
absolute sigma b6 = 0.029830073372480774
relative sigma b6 = 0.02130991354064502
absolute sigma b7 = 0.03073757639431299

In [30]:
'''
From now on, we repeat the same process as before but using all the internal functions of the code. In this way it is better explained how the code works and how to use it.

- define useful qunatities
- map the names of the fields to the probes in the grid previously computed
- calculate the fiducial spectra to use for the covariance matrix
- load the grid using process_cl_data
- compute the derivatives using the Polynomial method
- compute the Covariance matrix (with noise, fsky etc)
- Mask the probes and ells that we want to exclude
- compute the Fisher

For more details, check the documentation of the functions in the code: they are all commented

'''
# define the ells of the spectra
lmax = 1500
lmin = 2
ls  = np.arange(2,lmax+1,1)
#\ells vector starting from 2

ls2 = np.insert(ls,[0,0],[0,0])
#\ells vector starting from zero. To build the covariance matrix it is useful to use ls2
normfactor0 = (ls2*(ls2+1))/(2*np.pi)

As_fid= 2.12605e-9
ns_fid=0.96
H0_fid=67.0
ombh2_fid=0.05*0.67**2
omch2_fid=0.268565*0.67**2
tau_fid= 0.058
logAss_fid = np.log(As_fid*1e10)
omh2_fid=ombh2_fid+omch2_fid
sigma8_fid = 0.816

b1_fid = 1.0997727037892875
b2_fid = 1.220245876862528
b3_fid = 1.2723993083933989
b4_fid = 1.316624471897739
b5_fid = 1.35812370570578
b6_fid = 1.3998214171814918
b7_fid = 1.4446452851824907
b8_fid = 1.4964959071110084
b9_fid = 1.5652475842498528
b10_fid = 1.7429859437184225

#define the fields involved (not the probes!)
fields = ['T', 'E', 'P', 'W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10'] #IF YOU INTRODUCE NEW W, CHECK THE RESCALING FOR PXWi
n_fields = len(fields)
fskies = [0.7, 0.7, 0.7, 0.36, 0.36, 0.36, 0.36, 0.36, 0.36, 0.36, 0.36, 0.36, 0.36]
n_probes = (n_fields*(n_fields+1))//2


#define the parameters involved

vars = ["Omegam","Omegab","sigma8","ns","h0",'b1', 'b2', 'b3', 'b4','b5', 'b6','b7','b8','b9','b10']
n_params = len(vars)

n_counts = ut.count_ws(fields)

fid_values = [0.32, 0.05, sigma8_fid, ns_fid, H0_fid, b1_fid, b2_fid, b3_fid,b4_fid,b5_fid,b6_fid,b7_fid,b8_fid,b9_fid,b10_fid]


# define the perturbation array
step = 0.01
epsilon = [-10*step,0*step, +10*step]





In [38]:

file_path = '/home/guglielmo/Desktop/Codes/guglielmo/GR_test4/fiducial/unlensed/fiducial.txt'  # Update with the correct path

column_mapping = ut.get_mapping(file_path, n_fields)


In [39]:
#get the probes and keys
keys = ut.get_keys(fields)
indici = ut.get_Gauss_keys_v3(fields)
path_unlens = '/home/guglielmo/Desktop/Codes/guglielmo/GR_test4/fiducial/unlensed/fiducial.txt'
path_lens = '/home/guglielmo/Desktop/Codes/guglielmo/GR_test4/fiducial/lensed/fiducial.txt'
clsdict = ut.extract_fiducial_cl_from_CAMB(path_unlens, path_lens, column_mapping, n_counts, ls)


/home/guglielmo/Desktop/Codes/Euclid_Forecast_code/Utilities.py:241: RuntimeWarning: invalid value encountered in divide
  newvalue = value / normfactor0


In [5]:




# Base path to your data files
base_path = '/home/guglielmo/Desktop/Codes'

# Directory structure, where {var} will be replaced by the variable name ('Omegam', 'Sigma8', etc.)
directory_structure = 'guglielmo/GR_test4/{var}'

epsilon = [-10*step,0*step, +10*step]
# Call the function to process the data
cl_data = ut.process_cl_data(vars, epsilon, column_mapping, ls, base_path, directory_structure)


Processing Data: 100%|██████████| 4095/4095 [00:13<00:00, 304.39it/s]


In [7]:
#Compute the derivatives (derivatives = with the l*(l+1) factor and with the CMB lensing as deflection angle, r_squared_data = R2 test of goodness of the fit, derivate_data_scaled = derivatives scaled with CMB lensing convergence, derivate_data_rescaled = derivatives rescaled with the scaling factor and lensing convergence). The last one is to be used for the Fisher computation (more details in the documentation of the function)
derivatives_data, r_squared_data, derivate_data_scaled, derivate_data_rescaled = dv.polynomial_derivative(cl_data, column_mapping, vars, fid_values, epsilon, scaling_factor=True)


Processing Omegam...


100%|██████████| 1499/1499 [00:15<00:00, 96.31it/s] 


Processing Omegab...


100%|██████████| 1499/1499 [00:15<00:00, 99.35it/s] 


Processing sigma8...


...

100%|██████████| 2999/2999 [00:29<00:00, 100.89it/s]


Processing b9...


100%|██████████| 2999/2999 [00:30<00:00, 99.50it/s] 


Processing b10...


100%|██████████| 2999/2999 [00:31<00:00, 96.54it/s] 


In [40]:
#Define the noise


filename = "/home/guglielmo/Desktop/Codes/Euclid_Forecast_code/new_noises.npz"

data = np.load(filename, allow_pickle=True,encoding="latin1")
check = data['planck'].item()
check2 = data['euclid'].item()
N_G = check2['G5-G5']
N_L = check2['L4-L4']
N_P_adimensionale = check['E-E']
N_T_adimensionale = check['T-T']

nlkk= np.loadtxt(os.path.join( '/home/guglielmo/Desktop/Codes/nlkk.dat'), usecols=1)

N_K = nlkk[:lmax+1] #lensing convergence noise


#Final dictionary of noises
noisedict = {'TxT':N_T_adimensionale, 'ExE':N_P_adimensionale, "PxP": N_K, 'W1xW1': N_G, 'W2xW2': N_G, 'W3xW3': N_G, 'W4xW4': N_G, 'W5xW5': N_G, 'W6xW6': N_G, 'W7xW7': N_G, 'W8xW8': N_G, 'W9xW9': N_G, 'W10xW10': N_G}

In [31]:

# Call the function with the choice to set crosses to the minimum of the two fields' fsky values
fskies = fm.compute_fskies(n_fields, fields, fskies, use_min_fsky_for_crosses=True)


In [41]:
#Actually build the covariance and the for cycle divides by the initial factor each element (necessarily!)
fsky = None
covariance = fm.sigma(n_fields,lmax,lmin,fsky,fskies,indici,clsdict, noisedict)
for i in range(lmin, len(ls2)):
    covariance[:,:,i] = (covariance[:,:,i])/(2*ls2+1)[i]


0 0 ['T' 'T' 'T' 'T']
0 1 ['T' 'T' 'T' 'E']
0 2 ['T' 'T' 'T' 'P']
0 3 ['T' 'T' 'T' 'W1']
0 4 ['T' 'T' 'T' 'W2']
0 5 ['T' 'T' 'T' 'W3']
0 6 ['T' 'T' 'T' 'W4']
0 7 ['T' 'T' 'T' 'W5']
0 8 ['T' 'T' 'T' 'W6']
0 9 ['T' 'T' 'T' 'W7']
0 10 ['T' 'T' 'T' 'W8']
0 11 ['T' 'T' 'T' 'W9']
0 12 ['T' 'T' 'T' 'W10']
0 13 ['T' 'T' 'E' 'E']
0 14 ['T' 'T' 'E' 'P']
0 15 ['T' 'T' 'E' 'W1']
0 16 ['T' 'T' 'E' 'W2']
0 17 ['T' 'T' 'E' 'W3']
0 18 ['T' 'T' 'E' 'W4']
0 19 ['T' 'T' 'E' 'W5']
0 20 ['T' 'T' 'E' 'W6']
0 21 ['T' 'T' 'E' 'W7']
0 22 ['T' 'T' 'E' 'W8']
0 23 ['T' 'T' 'E' 'W9']
0 24 ['T' 'T' 'E' 'W10']
0 25 ['T' 'T' 'P' 'P']
0 26 ['T' 'T' 'P' 'W1']
0 27 ['T' 'T' 'P' 'W2']
0 28 ['T' 'T' 'P' 'W3']
0 29 ['T' 'T' 'P' 'W4']
0 30 ['T' 'T' 'P' 'W5']
0 31 ['T' 'T' 'P' 'W6']
0 32 ['T' 'T' 'P' 'W7']
0 33 ['T' 'T' 'P' 'W8']
0 34 ['T' 'T' 'P' 'W9']
0 35 ['T' 'T' 'P' 'W10']
0 36 ['T' 'T' 'W1' 'W1']
0 37 ['T' 'T' 'W1' 'W2']
0 38 ['T' 'T' 'W1' 'W3']
0 39 ['T' 'T' 'W1' 'W4']
0 40 ['T' 'T' 'W1' 'W5']
0 41 ['T' 'T' 'W1' 'W6']

In [42]:
lminimi = {'TxT':2,
'TxE':2,
'TxP':8,
'TxW1':10,
'TxW2':10,
'TxW3':10,
'TxW4':10,
'TxW5':10,
'TxW6':10,
'TxW7':10,
'TxW8':10,
'TxW9':10,
'TxW10':10,
'ExE':2,
'ExP':8,
'ExW1':10,
'ExW2':10,
'ExW3':10,
'ExW4':10,
'ExW5':10,
'ExW6':10,
'ExW7':10,
'ExW8':10,
'ExW9':10,
'ExW10':10,
'PxP':8,
'PxW1':10,
'PxW2':10,
'PxW3':10,
'PxW4':10,
'PxW5':10,
'PxW6':10,
'PxW7':10,
'PxW8':10,
'PxW9':10,
'PxW10':10,
'W1xW1':10,
'W1xW2':10,
'W1xW3':10,
'W1xW4':10,
'W1xW5':10,
'W1xW6':10,
'W1xW7':10,
'W1xW8':10,
'W1xW9':10,
'W1xW10':10,
'W2xW2':10,
'W2xW3':10,
'W2xW4':10,
'W2xW5':10,
'W2xW6':10,
'W2xW7':10,
'W2xW8':10,
'W2xW9':10,
'W2xW10':10,
'W3xW3':10,
'W3xW4':10,
'W3xW5':10,
'W3xW6':10,
'W3xW7':10,
'W3xW8':10,
'W3xW9':10,
'W3xW10':10,
'W4xW4':10,
'W4xW5':10,
'W4xW6':10,
'W4xW7':10,
'W4xW8':10,
'W4xW9':10,
'W4xW10':10,
'W5xW5':10,
'W5xW6':10,
'W5xW7':10,
'W5xW8':10,
'W5xW9':10,
'W5xW10':10,
'W6xW6':10,
'W6xW7':10,
'W6xW8':10,
'W6xW9':10,
'W6xW10':10,
'W7xW7':10,
'W7xW8':10,
'W7xW9':10,
'W7xW10':10,
'W8xW8':10,
'W8xW9':10,
'W8xW10':10,
'W9xW9':10,
'W9xW10':10,
'W10xW10':10,
}

In [43]:
lmassimi = {'TxT':1500,
'TxE':1500,
'TxP':400,
'TxW1':200,
'TxW2':200,
'TxW3':200,
'TxW4':200,
'TxW5':200,
'TxW6':200,
'TxW7':200,
'TxW8':200,
'TxW9':200,
'TxW10':200,
'ExE':1500,
'ExP':400,
'ExW1':200,
'ExW2':200,
'ExW3':200,
'ExW4':200,
'ExW5':200,
'ExW6':200,
'ExW7':200,
'ExW8':200,
'ExW9':200,
'ExW10':200,
'PxP':400,
'PxW1':400,
'PxW2':400,
'PxW3':400,
'PxW4':400,
'PxW5':400,
'PxW6':400,
'PxW7':400,
'PxW8':400,
'PxW9':400,
'PxW10':400,
'W1xW1':750,
'W1xW2':750,
'W1xW3':750,
'W1xW4':750,
'W1xW5':750,
'W1xW6':750,
'W1xW7':750,
'W1xW8':750,
'W1xW9':750,
'W1xW10':750,
'W2xW2':750,
'W2xW3':750,
'W2xW4':750,
'W2xW5':750,
'W2xW6':750,
'W2xW7':750,
'W2xW8':750,
'W2xW9':750,
'W2xW10':750,
'W3xW3':750,
'W3xW4':750,
'W3xW5':750,
'W3xW6':750,
'W3xW7':750,
'W3xW8':750,
'W3xW9':750,
'W3xW10':750,
'W4xW4':750,
'W4xW5':750,
'W4xW6':750,
'W4xW7':750,
'W4xW8':750,
'W4xW9':750,
'W4xW10':750,
'W5xW5':750,
'W5xW6':750,
'W5xW7':750,
'W5xW8':750,
'W5xW9':750,
'W5xW10':750,
'W6xW6':750,
'W6xW7':750,
'W6xW8':750,
'W6xW9':750,
'W6xW10':750,
'W7xW7':750,
'W7xW8':750,
'W7xW9':750,
'W7xW10':750,
'W8xW8':750,
'W8xW9':750,
'W8xW10':750,
'W9xW9':750,
'W9xW10':750,
'W10xW10':750,
}

In [44]:
#lminimi = {"TT" : 2, "TE" : 2, "TG":2, "TK":2, "EE" : 2 ,"EG" : 2 , "EK":2, "GG": 10, "GK":2, "KK":2 }
#lmassimi = {"TT" : 1500, "TE" : 1500, "TG":1500, "TK":1500, "EE" : 1500 ,"EG" : 1500 , "EK":1500, "GG": 750, "GK":1500, "KK":1500 }
excluded_probes_euclid = ['TxT',
'TxE',
'TxP',
'TxW1',
'TxW2',
'TxW3',
'TxW4',
'TxW5',
'TxW6',
'TxW7',
'TxW8',
'TxW9',
'TxW10',
'ExE',
'ExP',
'ExW1',
'ExW2',
'ExW3',
'ExW4',
'ExW5',
'ExW6',
'ExW7',
'ExW8',
'ExW9',
'ExW10',
'PxP',
'PxW1',
'PxW2',
'PxW3',
'PxW4',
'PxW5',
'PxW6',
'PxW7',
'PxW8',
'PxW9',
'PxW10',
 ]

In [45]:
masked_euclid = fm.get_masked_sigma(n_fields, lmin,lmax, indici, covariance,excluded_probes_euclid,lmins=lminimi,lmaxs=lmassimi )
inversa_euclid = fm.inv_sigma(lmin,lmax,masked_euclid)
masked_deriv_euclid = fm.get_masked_derivates(n_fields, masked_sigma=masked_euclid, derivative=derivate_data_rescaled,ells=ls)
fisher,covariance_fish_euclid,errori_euclid = fm.get_fisher(ls,n_params, masked_deriv_euclid, inversa_euclid)


TxT 2 1500
TxE 2 1500
TxP 8 400
TxW1 10 200
TxW2 10 200
TxW3 10 200
TxW4 10 200
TxW5 10 200
TxW6 10 200
TxW7 10 200
TxW8 10 200
TxW9 10 200
TxW10 10 200
ExE 2 1500
ExP 8 400
ExW1 10 200
ExW2 10 200
ExW3 10 200
ExW4 10 200
ExW5 10 200
ExW6 10 200
ExW7 10 200
ExW8 10 200
ExW9 10 200
ExW10 10 200
PxP 8 400
PxW1 10 400
PxW2 10 400
PxW3 10 400
PxW4 10 400
PxW5 10 400
PxW6 10 400
PxW7 10 400
PxW8 10 400
PxW9 10 400
PxW10 10 400
W1xW1 10 750
W1xW2 10 750
W1xW3 10 750
W1xW4 10 750
W1xW5 10 750
W1xW6 10 750
W1xW7 10 750
W1xW8 10 750
W1xW9 10 750
W1xW10 10 750
W2xW2 10 750
W2xW3 10 750
W2xW4 10 750
W2xW5 10 750
W2xW6 10 750
W2xW7 10 750
W2xW8 10 750
W2xW9 10 750
W2xW10 10 750
W3xW3 10 750
W3xW4 10 750
W3xW5 10 750
W3xW6 10 750
W3xW7 10 750
W3xW8 10 750
W3xW9 10 750
W3xW10 10 750
W4xW4 10 750
W4xW5 10 750
W4xW6 10 750
W4xW7 10 750
W4xW8 10 750
W4xW9 10 750
W4xW10 10 750
W5xW5 10 750
W5xW6 10 750
W5xW7 10 750
W5xW8 10 750
W5xW9 10 750
W5xW10 10 750
W6xW6 10 750
W6xW7 10 750
W6xW8 10 750
W6xW9 10 7

In [ ]:
errori_finali_euclid = ut.print_errors(vars, fid_values, errori_euclid)

DISCLAIMER: Omegab is ombh2 and Omegac is omch2
absolute sigma Omegam = 0.0054329900892157055
relative sigma Omegam = 0.01697809402879908
absolute sigma Omegab = 0.0022393808082702854
relative sigma Omegab = 0.044787616165405705
absolute sigma sigma8 = 0.007824981841225057
relative sigma sigma8 = 0.00958943853091306
absolute sigma ns = 0.009830873460511797
relative sigma ns = 0.010240493188033124
absolute sigma h0 = 1.9888416480327302
relative sigma h0 = 0.02968420370198105
absolute sigma b1 = 0.013942812002703253
relative sigma b1 = 0.012677903310986928
absolute sigma b2 = 0.01588015828975576
relative sigma b2 = 0.013013900387507564
absolute sigma b3 = 0.01681356615416507
relative sigma b3 = 0.013214064203944595
absolute sigma b4 = 0.015746741677900855
relative sigma b4 = 0.011959933917378904
absolute sigma b5 = 0.015879753600956243
relative sigma b5 = 0.011692420605164216
absolute sigma b6 = 0.016187724268010392
relative sigma b6 = 0.011564135302775981
absolute sigma b7 = 0.017635079